# Predicting methylation from annotation groups

In [1]:
import pandas as pd
import numpy as np

import os,subprocess
workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/modelling'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

In [2]:
annot=pd.read_csv('../explore_data/relevant_annotations.csv',sep='\t',header=None)
annot.columns=['id','Regulatory_Feature_Group','Relation_to_UCSC_CpG_Island',
    'Strand','Infinium_Design_Type','Random_Loci','Methyl27_Loci']
annot.fillna(0,inplace=True)
annot.head()

,id,Regulatory_Feature_Group,Relation_to_UCSC_CpG_Island,Strand,Infinium_Design_Type,Random_Loci,Methyl27_Loci
0,cg00035864,0,0,F,II,0,0
1,cg00050873,0,N_Shore,R,I,0,0
2,cg00061679,0,0,R,II,0,0
3,cg00063477,0,S_Shelf,F,II,0,0
4,cg00121626,0,N_Shore,R,II,0,0


In [3]:
labels=pd.read_csv('rf_test_preds.csv',sep='\t')[['id','label']]
labels.head()

,id,label
0,cg19752143,1
1,cg05219517,0
2,cg05218696,1
3,cg09329621,1
4,cg17608706,1


In [4]:
data=labels.merge(annot,on=['id'])
data.head()

,id,label,Regulatory_Feature_Group,Relation_to_UCSC_CpG_Island,Strand,Infinium_Design_Type,Random_Loci,Methyl27_Loci
0,cg19752143,1,0,0,F,II,0,0
1,cg05219517,0,Promoter_Associated,Island,R,I,0,0
2,cg05218696,1,0,0,F,II,0,0
3,cg09329621,1,0,0,R,II,0,0
4,cg17608706,1,Gene_Associated,N_Shelf,F,II,0,0


----

## Predict with naive Bayes

In [5]:
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.naive_bayes import GaussianNB

#### Only from cpg island group

In [6]:
lbenc=LabelEncoder()
lbenc.fit(data['Relation_to_UCSC_CpG_Island'].values)
x=lbenc.transform(data['Relation_to_UCSC_CpG_Island'].values)

lbenc=LabelBinarizer()
lbenc.fit(x)
x=lbenc.transform(x)

model=GaussianNB()
model.fit(x,data['label'].values)

pred=model.predict(x)
print 'accuracy:',list(labels['label']-pred).count(0)/float(len(pred))

accuracy: 0.80855


#### With reg feature group, though this is not easily learnable

In [7]:
lbenc=LabelEncoder()
x1=lbenc.fit_transform(data['Relation_to_UCSC_CpG_Island'].values)
x2=lbenc.fit_transform(data['Regulatory_Feature_Group'].values)

lbenc=LabelBinarizer()

x1=lbenc.fit_transform(x1)
x2=lbenc.fit_transform(x2)

x=np.column_stack([x1,x2])
model=GaussianNB()
model.fit(x,data['label'].values)

pred=model.predict(x)
print 'accuracy:',list(labels['label']-pred).count(0)/float(len(pred))

accuracy: 0.8648
